In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("resumes_filtered.csv")

In [3]:
df.drop(columns=["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
df.head()

In [5]:
pd.to_datetime('today').normalize().date()

datetime.date(2024, 6, 18)

In [6]:
pd.to_datetime(df["birth_date"][0]).date()

datetime.date(1985, 7, 30)

In [7]:
df["age"] = (pd.to_datetime('today').normalize() - pd.to_datetime(df["birth_date"])).dt.days // 365

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19820 entries, 0 to 19819
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   resume_id          19820 non-null  int64  
 1   contact_email      19820 non-null  object 
 2   is_male            19550 non-null  object 
 3   birth_date         19801 non-null  object 
 4   salary_from        14238 non-null  float64
 5   city_name          19820 non-null  object 
 6   is_married         7493 non-null   object 
 7   driving_license    4080 non-null   object 
 8   comment            5220 non-null   object 
 9   comment2           7413 non-null   object 
 10  organization_name  10595 non-null  object 
 11  working_date_from  11091 non-null  object 
 12  working_date_to    8240 non-null   object 
 13  position           11052 non-null  object 
 14  skills             16930 non-null  object 
 15  wanted_position    19820 non-null  object 
 16  age                198

In [9]:
df["is_male"].fillna("не указано", inplace=True)
df["birth_date"].fillna("не указано", inplace=True)
df["age"].fillna("не указано", inplace=True)
df["salary_from"].fillna(0.0, inplace=True)
df["is_married"].fillna("не указано", inplace=True)
df["driving_license"].fillna("отсутствует", inplace=True)
df.drop(columns=["comment", "comment2"], axis=1, inplace=True)
df["organization_name"].fillna("не указано", inplace=True)
df["working_date_from"].fillna("не указано", inplace=True)
df["working_date_to"].fillna("не указано", inplace=True)
df["position"].fillna("не указано", inplace=True)
df["skills"].fillna("не указано", inplace=True)


C:\Users\yxtag\AppData\Local\Temp\ipykernel_11068\3327791111.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'не указано' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["age"].fillna("не указано", inplace=True)


In [10]:
df["is_male"].unique()

array([True, False, 'не указано'], dtype=object)

In [ ]:
df["is_male"].replace({True: "мужчина", False: "женщина"}, inplace=True)
df.head()

In [12]:
df = df.drop_duplicates()

In [13]:
df["skills"] = df["skills"].apply(lambda x: x.replace('"', "").replace("[", "").replace("]", ""))

In [14]:
df["driving_license"] = df["driving_license"].apply(lambda x: x.replace('{', "").replace("}", ""))

In [15]:
df.drop(columns=["is_married"], axis=1, inplace=True)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5765 entries, 0 to 19819
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   resume_id          5765 non-null   int64  
 1   contact_email      5765 non-null   object 
 2   is_male            5765 non-null   object 
 3   birth_date         5765 non-null   object 
 4   salary_from        5765 non-null   float64
 5   city_name          5765 non-null   object 
 6   driving_license    5765 non-null   object 
 7   organization_name  5765 non-null   object 
 8   working_date_from  5765 non-null   object 
 9   working_date_to    5765 non-null   object 
 10  position           5765 non-null   object 
 11  skills             5765 non-null   object 
 12  wanted_position    5765 non-null   object 
 13  age                5765 non-null   object 
dtypes: float64(1), int64(1), object(12)
memory usage: 675.6+ KB


In [ ]:
df.head(20)

In [18]:
df["resume_id"].nunique()

5000

In [19]:
source = []
last_tmp_text = ""
for res_id in df["resume_id"].unique():

    # source
    source_text = ""

    # work expirience features
    work_exp = " Опыт работы:"

    org_name = ""
    date_s = ""
    date_f = ""
    pos = ""

    # general features
    gen_feat = ""

    sex = ""
    salary = ""
    bd = ""
    city = ""
    driving_license = ""
    skills = ""
    age=""
    
    last_view_vacs = ""
    wanted_pos = ""
    
    # get actual rows and sort
    rows = df[df["resume_id"] == res_id]
    rows = rows.sort_values("working_date_from")

    # work exp counter
    rows_count = 1

    for index, row in rows.iterrows():

        # work exp temp text
        tmp_text = ""
        
        # get general features
        if not any([sex, salary, bd, city, driving_license, skills]):
            sex = row["is_male"]
            salary = row["salary_from"]
            bd = row["birth_date"]
            city = row["city_name"]
            driving_license = row["driving_license"]
            skills = row["skills"]
        
        if not wanted_pos: wanted_pos = row["wanted_position"]
        if not age: age = row["age"]
        
        # get one place work expirience
        org_name = row["organization_name"]
        date_s = row["working_date_from"]
        date_f = row["working_date_to"]
        pos = row["position"]

        # create one place work exp text
        if pos == "не указано" and org_name == "не указано" and \
            date_s == "не указано" and date_f == "не указано":
            tmp_text = " без опыта работы"
            work_exp += tmp_text
            break
        else:
            tmp_text = " " + str(rows_count) + "." + " Место работы: " + str(org_name) + ". Дата начала работы: " + str(date_s) + \
                ". Дата окончания работы: " + str(date_f) + ". Должность: " + str(pos) + "."
        
            if last_tmp_text != tmp_text[3:]:
                work_exp += tmp_text

            last_tmp_text = tmp_text[3:]

            rows_count += 1
    if type(age) == float:
        age = int(age)
    gen_feat_text = "Пол: " + str(sex) + \
        ". Возраст: " + str(age) + " лет"\
        ". Желаемая зарплата: от " + str(salary) + " рублей" + \
        ". Желаемая должность: " + str(wanted_pos) + \
        ". Место проживания: " + str(city) + \
        ". Водительское удостоверение: " + str(driving_license) + \
        ". Умения/навыки: " + str(skills) + "."
    
    source_text = gen_feat_text + work_exp
    
    source.append(source_text)

In [ ]:
source

In [21]:
df_src = pd.DataFrame(data={"source": source})

In [ ]:
df_src["source"] = df_src["source"].replace('"', "")

In [23]:
df_src.to_csv("dpo_requests.csv")